# Expected Pocket Time
This notebook will compute the expected pocket time for each play within the full dataset.

## Overview
There are 2 neural networks being trained here, the first one (Model A) is designed to predict a player's movement in the next frame of a play given the current state in the frame being looked at. The implication of this is that the predicted movement represents an average movement that specific player will take in his current situation. Given that the model is only trained on the data we currently have, this is not a seasonal average. 

The second neural network is one that predicts the total time left the QB has in the pocket. This model is trained ONLY on sack plays where the total time in the pocket was reached. It is trained on a frame-by-frame basis (hence a current field state) and will compute the total seconds remaining that the QB has in the pocket given that field state.

All together when both models are being used, we can decompose the changes in pocket time remaning as the play moves forward in time. The jumps in the pocket time remaning are likely caused by an incident that happened on the field relative to either the offensive line or defensive line. We can then compute the average movement the O-line and D-line make given their field states to produce an average change pocket time. This average can be subtracted from the actual value to show whether each player is above or below their average in gaining or loosing the QB's time in the pocket.


In [4]:
import numpy as np
import pickle
from tqdm import tqdm
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

#Set by user -> location of the plays dictionary
data_dir = "/Users/admin/Desktop/data_bowl/data_bowl_github/"

#Open the dictionary
f = open(data_dir+'full_plays_dictionary.pckl', 'rb')
dat = pickle.load(f)
f.close()

In [5]:
#Set a list of functions needed to compute this
def GetClosePlayers(dat_dict,player,plID,frID,radius=250):
    '''
    This function will return a dictionary of the all players on the field from closest
    to farthest from the player in specified in the inputs.
    
    INPUTS:
        dat_dict -> dict. the plays dictionary
        player -> str. the player ID
        plID -> str. the play ID
        frID -> str. the frame ID
        
    RETURNS:
        A sorted dictionary from closest to farthest players relative to
        the input player.
    '''
    
    #Open the dictionary
    curr=dat_dict['pass'][plID][frID]
    ofIDs=list(curr.keys()) #List of all player IDs for this game ID and frame ID
    cx0=float(curr[player][0]) #X-position
    cy0=float(curr[player][1]) #Y-position
    
    #Create empty 1D arrays to be filled (distance, x-dist, y-dist, ID, possession)
    distv = np.zeros(len(ofIDs)) * np.nan
    xrelv=np.zeros(len(ofIDs)) * np.nan
    yrelv=np.zeros(len(ofIDs)) * np.nan
    idv=np.zeros(len(ofIDs)) * np.nan
    pos = np.zeros(len(ofIDs)).astype(object) * np.nan
    
    for count, idd in enumerate(ofIDs):
        if idd=='-999.0': #Skip over the football itself
            continue
        #Take the difference in x and y position from the specified player to the others
        xrel=float(curr[idd][0])-cx0
        yrel=float(curr[idd][1])-cy0
        dist_tot=np.sqrt(xrel**2+yrel**2) #Distance formula
        poss = curr[idd][-3]
        if dist_tot>radius:
            continue
        #Fill the arrays
        distv[count] = dist_tot
        idv[count] = idd
        xrelv[count] = xrel
        yrelv[count] = yrel
        pos[count] = poss

    #Sort the arrays here
    sortperm=np.argsort(distv)
    idv = idv[sortperm]
    distv = distv[sortperm]
    yrelv = yrelv[sortperm]
    xrelv = xrelv[sortperm]
    pos = pos[sortperm]
    
    return({'idv':idv,'distv':distv,'xrelv':xrelv,'yrelv':yrelv, 'pos': pos})

def os2xyspeed(o,s):
    '''
    This function will take in the orientation of the player and return their x,y triangular coordinates.
    INPUTS:
        o,s -> 1D arrays of orientation and speed
    RETURNS:
        x,y -> arrays of input dimension corresponding to the 
        triangulated coordinates.
    '''
    
    if o<90:
        ot=o*2*np.pi/360
        x=s*np.sin(ot)
        y=s*np.cos(ot)
    elif o<180:
        ot=(180-o)*2*np.pi/360
        x=s*np.sin(ot)
        y=-s*np.cos(ot)
    elif o<180:
        ot=(o-180)*2*np.pi/360
        x=-s*np.sin(ot)
        y=-s*np.cos(ot)
    else:
        ot=(360-o)*2*np.pi/360
        x=-s*np.sin(ot)
        y=s*np.cos(ot)
    return([x,y])

def GetFieldStateFeatures(plID,frID,dat_dict):
    '''
    This function will analyze the current state of the field and produce arrays that are meant to 
    train a neural network.
    
    INPUTS:
        plID -> str. the play ID
        frID -> str. the frame ID
        dat_dict -> the plays dictionary
        
    RETURNS:
        x -> list of arrays corresponding to the field state
        x_players -> list of arrays corresponding to the player ID in x
    '''
    
    plID,frID = str(plID),str(frID)
    
    teamkick=play_d[plID][4] #Find the posession Team
    homeACR=game_d[plID.split("-")[0]][4] #Find Home Team
    awayACR=game_d[plID.split("-")[0]][5] #Find Away Team
    if teamkick==homeACR:
        homekick=True
    else:
        homekick=False
#-------------------------------------------------------------------------------------------------------------------
    #reverse x coordinates for all objects if playdirection it to the right
    revx=False
    if dat_dict['pass'][plID]['1']['-999.0'][8].strip("\"")=='right':
        revx=True
#-------------------------------------------------------------------------------------------------------------------
    gcpout=GetClosePlayers(dat,'-999.0',plID,frID,radius=1000) #Outputs all players data in relation to the ball
    
    idx = np.where(gcpout['pos'] == 'QB')[0][0]
    bholder = str(gcpout['idv'][idx]) #Get Player ID of the QB
    
    bx=float(dat_dict['pass'][plID][frID]['-999.0'][0]) #X-Coordinate of the ball
    by=float(dat_dict['pass'][plID][frID]['-999.0'][1]) #Y-Coordinate of the ball
    cx=float(dat_dict['pass'][plID][frID][bholder][0]) #X-Coordinate of the QB
    cy=float(dat_dict['pass'][plID][frID][bholder][1]) #Y-Coordinate of the QB
    cs=float(dat_dict['pass'][plID][frID][bholder][2]) #Speed of the QB at the given frame
    ca=float(dat_dict['pass'][plID][frID][bholder][3]) #Acceleration of the QB at the given frame
    cdis=float(dat_dict['pass'][plID][frID][bholder][4]) #Distance Traveled by the QB between last frame and current frame
    co=float(dat_dict['pass'][plID][frID][bholder][5]) #Orientation of the QB 
    temp=os2xyspeed(co,cs)
    cxspd=temp[0] #X-Velocity of the QB
    cyspd=temp[1] #Y-Velocity of the QB
    if revx:
        cxspd=-cxspd #If Play was to the right, flip X-Coordinates
        bx=120-bx
        cx=120-cx
    x=[]
    x_players = []
    #------------------------------------------------------------------------------------------------------------------- 
    
    #Iterate through all players now
    for i in range(len(gcpout['idv'])): 
        cID=str(gcpout['idv'][i]) 
        if cID !='nan':
            
            cx=float(dat_dict['pass'][plID][frID][cID][0])
            cy=float(dat_dict['pass'][plID][frID][cID][1])
            cs=float(dat_dict['pass'][plID][frID][cID][2])
            ca=float(dat_dict['pass'][plID][frID][cID][3])
            cdis=float(dat_dict['pass'][plID][frID][cID][4])
            co=float(dat_dict['pass'][plID][frID][cID][5])# relx, rely, dist, xspeed, yspeed, acceleration, kicking_team_boolean, SORT BY DISTANCE TO BALL
            temp=os2xyspeed(co,cs)
            cxspd=temp[0]
            cyspd=temp[1]
            if revx:
                cxspd=-cxspd
                cx=120-cx
            cretind=0
            if (homekick and dat_dict['pass'][plID][frID][cID][11].strip("\"")=="home") or ((not homekick) and dat_dict['pass'][plID][frID][cID][11].strip("\"")=="away"):
                cretind=1
            x+=[(cx-bx)/20,(cy-by)/20,gcpout['distv'][i]/13,cxspd,cyspd,ca,cretind]
            x_players+= [cID]*7

    return(x,x_players)

def GetClosePlayers_5(dat_dict,player,plID,frID,radius=250):
    '''
    Similar to the function GetClosePlayers, this will return the first 5 players.
    
    INPUTS:
        dat_dict -> dict. containing the full dataset
        player -> str. the player ID
        plID -> str. the play ID
        frID -> str. the frame ID
   
    RETURNS:
        dictionary containing the first 5 players closest to the player specified.
    '''
    #Read in the dictionary
    curr=dat_dict['pass'][plID][frID]
    ofIDs=list(curr.keys())
    cx0=float(curr[player][0])
    cy0=float(curr[player][1])
    
    #Create empty arrays to be filled (distance, x-dist, y-dist, ID, possesion)
    distv = np.zeros(len(ofIDs)) * np.nan
    xrelv=np.zeros(len(ofIDs)) * np.nan
    yrelv=np.zeros(len(ofIDs)) * np.nan
    idv=np.zeros(len(ofIDs)) * np.nan
    pos = np.zeros(len(ofIDs)).astype(object) * np.nan
    
    #Loop through each player ID
    for count, idd in enumerate(ofIDs):
        if idd!='-999.0': #remove the football location, no need for that
            xrel=float(curr[idd][0])-cx0
            yrel=float(curr[idd][1])-cy0
            dist_tot=np.sqrt(xrel**2+yrel**2)
            poss = curr[idd][-3]
            if dist_tot<radius:
                distv[count] = dist_tot
                idv[count] = idd
                xrelv[count] = xrel
                yrelv[count] = yrel
                pos[count] = poss

    #Sort the data here
    sortperm=np.argsort(distv)
    idv = idv[sortperm]
    distv = distv[sortperm]
    yrelv = yrelv[sortperm]
    xrelv = xrelv[sortperm]
    pos = pos[sortperm]  
    
    return({'idv':idv,'distv':distv,'xrelv':xrelv,'yrelv':yrelv, 'pos': pos})

def testerrv(net,testloader):
    '''
    This function tests the trained neural network.
    
    INPUTS:
        net -> the model object
        testloader -> the testing set
    RETURNS:
        the correlation between the testing and training set of the neural networks performance
        
    '''
    outerr=[]
    testout=np.array([])
    trainout=np.array([])
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            
            #Calculate outputs by running images through the network
            images=images.float()
            labels=labels.float()
            outputs = net(images).squeeze()
            
            #The class with the highest energy is what we choose as prediction
            runtot=0
            denom=float(labels.shape[0])
            for i in range(int(denom)):
                runtot+=abs(labels[i]-outputs[i])/denom
            outerr.append(float(runtot))
            testout=np.append(testout,torch.Tensor.numpy(outputs))
            trainout=np.append(trainout,torch.Tensor.numpy(labels))
    
    return(np.corrcoef(testout,trainout)[0,1])

def model_a_assimilation(dat_dict,plID,frID,frIDs,j,playerID):
    '''
    This function assimates and trains the first round of neural networks.
    
    INPUTS:
        dat_dict -> dict. the full plays dictionary
        plID -> str. the play ID
        frID -> str. the frame ID
        frIDs -> list of all frame IDs for this play
        j -> int. an iterator
        playerID -> str. the player ID
        
    RETURNS:
        curx -> list containing the predictor variables needed to predict the movement 
        of the player specified given the current state of the field.
    
    '''
    
    teampos = play_d[plID][4] #Finds posession Team
    homeACR = game_d[plID.split("-")[0]][4] #Finds Home Team
    awayACR = game_d[plID.split("-")[0]][5] #Finds Away Team
    if teampos == homeACR:
        homepos = True
    else:
        homepos = False
    
    #Get the outcome vector: [deltax,deltay,a,xspd,yspd]
    cx = float(dat_dict['pass'][plID][frID][playerID][0])
    cy = float(dat_dict['pass'][plID][frID][playerID][1])
    cs = float(dat_dict['pass'][plID][frID][playerID][2])
    ca = float(dat_dict['pass'][plID][frID][playerID][3])
    cdis = float(dat_dict['pass'][plID][frID][playerID][4])
    co = float(dat_dict['pass'][plID][frID][playerID][5])
    nx = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][0])
    ny = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][1])
    ns = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][2])
    na = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][3])
    ndis = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][4])
    no = float(dat_dict['pass'][plID][frIDs[j+1]][playerID][5])
    temp = os2xyspeed(no,ns)
    nxspd = temp[0]
    nyspd = temp[1]
    temp = os2xyspeed(co,cs)
    cxspd = temp[0]
    cyspd = temp[1]
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
    ### get 5 closest players to the player specified
    gcpout=GetClosePlayers(dat,playerID,plID,frID,radius=250)
#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------

    #X-VARIABLE
    curx=[ca,cxspd,cyspd,cdis] ####(PREDICTOR) Current Field State for reference player####
#-------------------------------------------------------------------------------------------------------------------     
    for l in range(len(gcpout)):
        ID2=str(gcpout['idv'][l])
        cx2=float(dat_dict['pass'][plID][frID][ID2][0])
        cy2=float(dat_dict['pass'][plID][frID][ID2][1])
        cs2=float(dat_dict['pass'][plID][frID][ID2][2])
        ca2=float(dat_dict['pass'][plID][frID][ID2][3])
        cdis2=float(dat_dict['pass'][plID][frID][ID2][4])
        co2=float(dat_dict['pass'][plID][frID][ID2][5])
        temp=os2xyspeed(co2,cs2)
        cxspd2=temp[0]
        cyspd2=temp[1]

        #sameteam = 1 if reference player and closest player in question is on the same team
        if dat_dict['pass'][plID][frID][ID2][11]==dat_dict['pass'][plID][frID][playerID][11]:
            sameteam=1
        else:
            sameteam=0

        #(PREDICTOR) Adds distance, difference in x&y coords, x&y speed, 
        #same team indicator for all players (including reference player)
        curx+=[np.sqrt((cx2-cx)**2+(cy2-cy)**2),cx2-cx,cy2-cy,ca2,cxspd2,cyspd2,sameteam]
#-------------------------------------------------------------------------------------------------------------------
    # (PREDICTOR) Adds x&y coords of the ball
    cx2=float(dat_dict['pass'][plID][frID]['-999.0'][0])
    cy2=float(dat_dict['pass'][plID][frID]['-999.0'][1])

    #Indicator to determine who is posession team
    cretind=0
    if (homepos and dat_dict['pass'][plID][frID][playerID][11].strip("\"")=="home") or ((not homepos) and dat_dict['pass'][plID][frID][playerID][11].strip("\"")=="away"):
        cretind=1
#-------------------------------------------------------------------------------------------------------------------            
    #(PREDICTOR) Adds Footabll x&y coords, posession team indicator (1 = posession, 0 = defending)
    curx+=[cx2,cy2,cretind]

    return curx

def player_pos2player_id(dat_dict,gameId,Player_Pos):
    '''
    This function will return the player ID for the specified position.
    
    INPUTS:
        dat_dict -> dict. the full plays dictionary
        gameID -> str. the game ID
        Player_Pos -. str. the player position
        
    RETURNS:
        pid -> list of the player IDs corresponding to that position
    '''
    
    #Get the list of all player IDs
    player_IDs_dict = list(dat_dict['pass'][gameId]['1'].keys())
    pid = []
    for p in player_IDs_dict:
        if dat_dict['pass'][games[i]]['1'][str(p)][-3] == Player_Pos:
            pid.append(p)
    return pid


#Initiate the neural network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(154, 200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,20)
        self.fc4 = nn.Linear(20, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)#.squeeze()
        return x
net = Net()


In [ ]:
import pandas as pd
2021090900-2330
WEEK = '1'
#d = pd.read_csv(data_dir+"raw_data/week" +WEEK+ ".csv")
np.unique(d.loc[np.where((d.gameId==2021090900)&(d.playId==2298))[0]].event.values)

In [6]:
#Organize the data to ingest into the model
os.chdir(data_dir)

WEEKS = ['1','2','3','4','5','6','7','8']
catchframe = {}
passframe = {}
sackframe = {}
pID_s=set()
pID_p=set()
pID_sa=set()

for WEEK in WEEKS:
    filein=open(data_dir+"raw_data/week" +WEEK+ ".csv",'r')
    line=filein.readline()

    #Get all playIDs when ball was snapped and onward
    filein.seek(0)
    for line in filein:
        line_v=line.split(",")
        if line_v[15].strip('\n').strip('""') == 'ball_snap' or line_v[15].strip('\n').strip('""') =="autoevent_ballsnap":
            gpID = line_v[0]+"-"+line_v[1]
            pID_s.add(gpID)
            catchframe[gpID] = line_v[3]
            
    #Get all playIDs when ball was thrown and onward
    filein.seek(0)
    for line in filein:
        line_v=line.split(",")
        if line_v[15].strip('\n').strip('""') == 'pass_forward' or line_v[15].strip('\n').strip('""') =="autoevent_passforward" or line_v[15].strip('\n').strip('""') =="qb_sack" or line_v[15].strip('\n').strip('""') =="qb_strip_sack":
            gpID = line_v[0]+"-"+line_v[1]
            pID_p.add(gpID)
            passframe[gpID] = line_v[3]
            
    #Get all playIDs if the Qb was sacked
    filein.seek(0)
    for line in filein:
        line_v=line.split(",")
        if line_v[15].strip('\n').strip('""') == 'qb_sack':
            gpID = line_v[0]+"-"+line_v[1]
            pID_sa.add(gpID)
            sackframe[gpID] = line_v[3]


filein2=open(data_dir+"raw_data/games.csv",'r')
filein2.readline()
game_d={}
for line in filein2:
    line_v=[x.strip('\"') for x in line.strip("\n").split(",")]
    game_d[line_v[0]]=line_v[1:]

filein3=open(data_dir+"raw_data/plays.csv",'r')
filein3.readline()
play_d={}
for line in filein3:
    line_v=[x.strip('\"') for x in line.strip("\n").split(",")]
    play_d[line_v[0]+"-"+line_v[1]]=line_v[2:]
    


In [7]:
#Create a second neural network that is trained ONLY on sack plays (where the true pocket time was reached)
games = list(pID_sa)
frames_tot = np.sum(np.array([len(list(dat['pass'][games[i]].keys())) for i in range(len(games))]))
Y = np.zeros((frames_tot))*np.nan #[playId,seconds remaining]
X = np.zeros((frames_tot,22*7))*np.nan #[playId, stats for each player]

count = 0
for i in tqdm(range(len(games))):
    #Get the frame at which the QB was sacked
    try:
        frames = list(dat['pass'][games[i]].keys())

        #Get the frame at which the ball was snapped
        ball_snap = int(catchframe[games[i]])
        #ball_pass = int(passframe[games[i]])
        sack_frame = int(sackframe[games[i]])
    except KeyError:
        sack_frame = int(sackframe[games[i]])
        
    tot_frames = frames[ball_snap:sack_frame]
    for j in range(len(tot_frames)): #up until QB was sacked (including)
        X[count,:] = GetFieldStateFeatures(games[i],tot_frames[j],dat)[0]
        Y[count] = ((int(sack_frame)-int(tot_frames[j]))*0.1)
        
        count += 1
        
#Shave off frames after the sack
ishave = np.where(np.isnan(Y))[0][0]
X,Y = X[:ishave],Y[:ishave]

xt = torch.tensor(X)
yt = torch.tensor(Y)
trainset=torch.utils.data.TensorDataset(xt[:int(len(xt)*.9)],yt[:int(len(xt)*.9)])
testset=torch.utils.data.TensorDataset(xt[int(len(xt)*.9):],yt[int(len(xt)*.9):])

batch_size=64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=True, num_workers=0)

#Train the model
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)#, momentum=0.95)
testcor=[]
traincor=[]
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs=inputs.float()
        labels=labels.float()
        optimizer.zero_grad()
        outputs = net(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
    testerr=testerrv(net,testloader)
print('Finished Training') #4.8 loss



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 451/451 [00:05<00:00, 84.54it/s]


Finished Training


In [25]:
#Use both models on all plays to compute the expected pocket time
games = list(dat['pass'].keys())
return_dict = {}
for i in tqdm(range(len(games))):
    try:
        return_dict[str(games[i])] = {}
        return_dict[str(games[i])]['EPT_ave'] = {}
        return_dict[str(games[i])]['EPT_decomp'] = {}

        #Create arrays for the function
        frames_tot = np.sum(np.array([len(list(dat['pass'][games[i]].keys())) for i in range(len(games))]))
        X = np.zeros((frames_tot,22*7))*np.nan #[playId, stats for each player]
        X_players = np.zeros((frames_tot,22*7))*np.nan


        count = 0
        #Get the frame at which the QB was sacked
        frames = list(dat['pass'][games[i]].keys())

        #Get the frame at which the ball was snapped
        ball_snap = int(catchframe[games[i]])
        ball_pass = int(passframe[games[i]])

        tot_frames = frames[ball_snap:ball_pass+1]
        for j in range(len(tot_frames)-1): #up until QB was sacked (including)
            playerID_DE = player_pos2player_id(dat,games[i],'DE')

            if '43292.0' in playerID_DE:
                print('yes')
            count += 1
    except KeyError:
        print(games[i])
        
        

  1%|▌                                                                                                                  | 45/8555 [00:24<1:14:01,  1.92it/s]

2021090900-2298


  1%|▌                                                                                                                  | 46/8555 [00:24<1:14:17,  1.91it/s]

2021090900-2330


  1%|█▍                                                                                                                | 110/8555 [00:58<1:15:39,  1.86it/s]

...

2021092605-3296


 32%|████████████████████████████████████▌                                                                              | 2722/8555 [24:18<51:40,  1.88it/s]

2021092605-3684


 32%|████████████████████████████████████▋                                                                              | 2733/8555 [24:25<52:00,  1.87it/s]


KeyboardInterrupt: 

In [16]:
#Use both models on all plays to compute the expected pocket time
games = list(dat['pass'].keys())
return_dict = {}
for i in tqdm(range(len(games))):
    i = 44
    #try:
    return_dict[str(games[i])] = {}
    return_dict[str(games[i])]['EPT_ave'] = {}
    return_dict[str(games[i])]['EPT_decomp'] = {}

    #Create arrays for the function
    frames_tot = np.sum(np.array([len(list(dat['pass'][games[i]].keys())) for i in range(len(games))]))
    X = np.zeros((frames_tot,22*7))*np.nan #[playId, stats for each player]
    X_players = np.zeros((frames_tot,22*7))*np.nan


    count = 0
    #Get the frame at which the QB was sacked
    frames = list(dat['pass'][games[i]].keys())

    #Get the frame at which the ball was snapped
    ball_snap = int(catchframe[games[i]])
    ball_pass = int(passframe[games[i]])

    tot_frames = frames[ball_snap:ball_pass+1]
    for j in range(len(tot_frames)-1): #up until QB was sacked (including)
        X[count,:],X_players[count,:] = GetFieldStateFeatures(games[i],tot_frames[j],dat)

        #Get offsenive linemam
        playerID_C = player_pos2player_id(dat,games[i],'C')
        playerID_G = player_pos2player_id(dat,games[i],'G')
        playerID_T = player_pos2player_id(dat,games[i],'T')

        #Get defensive lineman
        playerID_NT = player_pos2player_id(dat,games[i],'NT')
        playerID_DT = player_pos2player_id(dat,games[i],'DT')
        playerID_DE = player_pos2player_id(dat,games[i],'DE')

        if '43292.0' in playerID_DE:
            print('yes')

        all_players = np.concatenate((playerID_C,playerID_G,playerID_T,playerID_NT,playerID_DT,playerID_DE))

        #Initialize empty arrays
        if j == 0:
            playerID = np.zeros((frames_tot,all_players.size))
            X_a = np.zeros((all_players.size,frames_tot,6*7))*np.nan #[O-line player,playId, 5 closest players + reference player w/ 7 data points]

        playerID[count,:] = all_players
        playerID_tags = np.concatenate((['C']*len(playerID_C),['G']*len(playerID_G),['T']*len(playerID_T),['NT']*len(playerID_NT),['DT']*len(playerID_DT),['DE']*len(playerID_DE)))[:6]

        X_a[:,count,:] = [model_a_assimilation(dat,games[i],tot_frames[j],tot_frames,j,str(pid)) for pid in playerID[count,:]]

        count += 1
    #Remove nan data points (frames after the ball was thrown)
    ishave = np.where(np.isnan(X))[0][0]
    X = X[:ishave]
    X_players = X_players[:ishave]
    X_a = X_a[:,:ishave,:]
    xt = torch.tensor(X)

    #Call the model and make the prediction
    pred=np.zeros((X.shape[0]))*np.nan

    with torch.no_grad():
        for count,data in enumerate(xt):
            outputs = net(data.float()).squeeze()
            pred[count] = float(outputs)

    return_dict[str(games[i])]['EPT'] = pred

    #Put the field states into model a for the O-line players to find contributions to the total expected time
    net_a = torch.load(data_dir+'model_a_full.pt') #model a
    xt_a = torch.tensor(X_a)

    #Call the model and make the prediction for each O-line player
    pred_a=np.zeros((X_a.shape[0],X_a.shape[1],5))*np.nan #[O-line Player, frames, (dx,dy,a,u,v)]
    for c in range(X_a.shape[0]):
        with torch.no_grad():
            for count,data in enumerate(xt_a[c]):
                outputs = net_a(data.float()).squeeze()
                pred_a[c,count,:] = np.array(outputs).astype(float)


    #Compute the field states needed for model B from model A's output a: (dx,dy,a,u,v) b: (xball,yball,dist,u,v,a)
    playerID = playerID[0].tolist()
    pred_newb_return = []
    for i_ in range(pred_a.shape[0]):
        X_newb = np.copy(X)
        for j in range(X_newb.shape[0]):
            #dx relative to the football
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][0]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][0]] + pred_a[i_][j][0]

            #dy relative to the football
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][1]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][1]] + pred_a[i_][j][1]

            #distance relative to the football
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][2]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][2]] + (np.sqrt((pred_a[i_][j][0]**2)+(pred_a[i_][j][1]**2)))

            #u component of the speed
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][3]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][3]] + pred_a[i_][j][3]

            #v component of the speed
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][4]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][4]] + pred_a[i_][j][4]

            #Acceleration of the player
            X_newb[j][np.where(X_players[j]==float(playerID[i_]))[0][5]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][5]] + pred_a[i_][j][2]

        #Call the model and make the prediction of the contribution
        xt_newb = torch.tensor(X_newb)
        pred_newb=np.zeros((len(X_newb)))*np.nan

        with torch.no_grad():
            for count,data in enumerate(xt_newb):
                outputs = net(data.float()).squeeze()
                pred_newb[count] = float(outputs)
        return_dict[str(games[i])]['EPT_ave'][str(playerID[i_])] = pred_newb

    #Compute the field states needed for model B -> second controlled experiment
    for i_ in range(len(playerID)):
        X_newb2 = np.copy(X)
        for j in range(X_newb2.shape[0]-1):
            #dx relative to the football
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][0]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][0]] + X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][0]]

            #dy relative to the football
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][1]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][1]] + X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][1]]

            #distance relative to the football
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][2]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][2]] + (np.sqrt((X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][0]]**2)+(X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][1]]**2)))

            #u component of the speed
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][3]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][3]] + X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][3]]

            #v component of the speed
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][4]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][4]] + X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][4]]

            #Acceleration of the player
            X_newb2[j][np.where(X_players[j]==float(playerID[i_]))[0][5]] = X[j][np.where(X_players[j]==float(playerID[i_]))[0][5]] + X[j+1][np.where(X_players[j+1]==float(playerID[i_]))[0][2]]


        #Call the model and make the prediction of the contribution
        xt_newb2 = torch.tensor(X_newb2)
        pred_newb2 = np.zeros((len(X_newb2)))*np.nan

        with torch.no_grad():
            for count,data in enumerate(xt_newb2):
                outputs = net(data.float()).squeeze()
                pred_newb2[count] = float(outputs)
        return_dict[str(games[i])]['EPT_decomp'][str(playerID[i_])] = pred_newb2
    #except KeyError:
        #pass


  0%|                                                                                                                              | 0/8555 [00:00<?, ?it/s]


KeyError: '2021090900-2298'

In [60]:
#Dump this data into a pickle file for later analysis
with open('EPT_variables.pkl', 'wb') as f:
    pickle.dump([return_dict], f)

In [52]:
return_dict[str(games[i])].keys()

dict_keys(['EPT_ave', 'EPT_decomp', 'EPT'])

In [66]:
dat['pass'][str(games[0])]['1']['25511.0']

['37.77',
 '24.22',
 '0.29',
 '0.3',
 '0.03',
 '165.16',
 '84.99',
 '12.0',
 'right',
 'QB',
 '-1.73852913314477',
 'home']

In [26]:
games[i]

'2021092605-4162'

In [27]:
ball_snap = int(catchframe['2021091913-2775'])
ball_pass = int(passframe['2021091913-2775'])
ball_snap,ball_pass

(6, 55)